## Data preprocessing

Before we can start building actual models using AzureML, we first have to prepare the data. In this hackathon we will focus on Predictive maintenance, and use a steel plates faults dataset. This dataset is retrieved here: https://archive.ics.uci.edu/ml/datasets/Steel+Plates+Faults. De csv file is saved in data/steel_plates_raw.csv.

The dataset contains data on a number of steel plate features and the fault observed. In total there's 7 types of faults. 

In Predictive maintenance, the goal is to predict faults/machine failures to prevent unwanted consequences such as machine downtime. To mock a PdM case, we will treat one of the 7 fault types as 'healthy' and try to build a model that accurately distinguishes between 'healthy' steel plates and broken ones. 

To be able to build this model, we also need healthy steel plates in our data. Since the data does not involve healthy observations, we will take one fault-type and treat this group as healthy. In order to represent a more normal situation, we will need a big group of healthy observations along with different smaller group of steel plate faults. Therefore, data was synthetically generated. However, these synthetic data requires some preprocessing to make it ready to use in our AzureML models. This notebook will touch on some basics around Working with Data in AzureML. For more information please refer to the Working with Data notebook from aml labs: https://github.com/MicrosoftDocs/mslearn-aml-labs


In [1]:
import os
import numpy as np
import pandas as pd
from azureml.core import Workspace, Experiment, Dataset

# Load the workspace
ws = Workspace.from_config()

# Load and view default datastore
# Datastores are references to storage locations such as Azure Storage blob containers
default_ds = ws.get_default_datastore()
for ds_name in ws.datastores:
    print(ds_name, "- Default =", ds_name == default_ds.name)

azureml_globaldatasets - Default = False
workspacefilestore - Default = False
workspaceblobstore - Default = True


In [3]:
# Upload the raw and synthesized csv files to the default datastore

default_ds.upload_files(files=['data/steel_plates_raw.csv', 'data/steel_plates_synthesized.csv'], 
                       target_path='pdm-data/', 
                       overwrite=True, 
                       show_progress=True)

Uploading an estimated of 2 files
Uploading data/steel_plates_raw.csv
Uploading data/steel_plates_synthesized.csv
Uploaded data/steel_plates_raw.csv, 1 files out of an estimated total of 2
Uploaded data/steel_plates_synthesized.csv, 2 files out of an estimated total of 2
Uploaded 2 files


$AZUREML_DATAREFERENCE_d617815090e74adabb64a8b8bd451219

### Working with Data in AzureML

AzureML offers different ways to work with data. One possibility is to create a <i>data reference</i>, which is a reference to a path in datastore that can be used in a script. Another possibility are <i>Datasets</i>, which can be either <i>Tabular Datasets</i> of <i>File Datasets</i>. Since the steel plates data is in structured format, we will use the tabular Dataset.

In [4]:
# Create Tabular dataset from uploaded csv in datastore
sp_raw_dataset = Dataset.Tabular.from_delimited_files(path=(default_ds, 'pdm-data/steel_plates_raw.csv'))


# Register the tabular dataset
try:
    sp_raw_dataset = sp_raw_dataset.register(workspace=ws, 
                                        name='steelplate raw dataset',
                                        description='steelplate raw faults',
                                        tags = {'format':'CSV'},
                                        create_new_version=True)
except Exception as ex:
    print(ex)

# Display first 10 rows    
sp_raw_dataset.take(10).to_pandas_dataframe()

,Column1,X_Minimum,X_Maximum,Y_Minimum,Y_Maximum,Pixels_Areas,X_Perimeter,Y_Perimeter,Sum_of_Luminosity,Minimum_of_Luminosity,...,Orientation_Index,Luminosity_Index,SigmoidOfAreas,Pastry,Z_Scratch,K_Scatch,Stains,Dirtiness,Bumps,Other_Faults
0,0,42,50,270900,270944,267,17,44,24220,76,...,0.8182,-0.2913,0.5822,1,0,0,0,0,0,0
1,1,645,651,2538079,2538108,108,10,30,11397,84,...,0.7931,-0.1756,0.2984,1,0,0,0,0,0,0
2,2,829,835,1553913,1553931,71,8,19,7972,99,...,0.6667,-0.1228,0.2150,1,0,0,0,0,0,0
3,3,853,860,369370,369415,176,13,45,18996,99,...,0.8444,-0.1568,0.5212,1,0,0,0,0,0,0
4,4,1289,1306,498078,498335,2409,60,260,246930,37,...,0.9338,-0.1992,1.0000,1,0,0,0,0,0,0
5,5,430,441,100250,100337,630,20,87,62357,64,...,0.8736,-0.2267,0.9874,1,0,0,0,0,0,0
6,6,413,446,138468,138883,9052,230,432,1481991,23,...,0.9205,0.2791,1.0000,1,0,0,0,0,0,0
7,7,190,200,210936,210956,132,11,20,20007,124,...,0.5000,0.1841,0.3359,1,0,0,0,0,0,0
8,8,330,343,429227,429253,264,15,26,29748,53,...,0.5000,-0.1197,0.5593,1,0,0,0,0,0,0
9,9,74,90,779144,779308,1506,46,167,180215,53,...,0.9024,-0.0651,1.0000,1,0,0,0,0,0,0


In [5]:
# Create Tabular dataset from uploaded csv in datastore
sp_synth_dataset = Dataset.Tabular.from_delimited_files(path=(default_ds, 'pdm-data/steel_plates_synthesized.csv'))


# Register the tabular dataset
try:
    sp_synth_dataset = sp_synth_dataset.register(workspace=ws, 
                                        name='steelplate synthesized dataset',
                                        description='steelplate synthesized faults',
                                        tags = {'format':'CSV'},
                                        create_new_version=True)
except Exception as ex:
    print(ex)

# Display first 10 rows    
sp_synth_dataset.take(10).to_pandas_dataframe()

,Column1,X_Minimum,X_Maximum,Y_Minimum,Y_Maximum,Pixels_Areas,X_Perimeter,Y_Perimeter,Sum_of_Luminosity,Minimum_of_Luminosity,...,Orientation_Index,Luminosity_Index,SigmoidOfAreas,Pastry,Z_Scratch,K_Scatch,Stains,Dirtiness,Bumps,Other_Faults
0,0,996.360295,1155.550369,1.138242e+06,1.854857e+06,312.560825,39.532720,35.122127,45013.495621,95.650313,...,0.838682,-0.345396,0.748645,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1,94.811225,214.082246,2.668494e+05,4.097364e+05,294.786049,14.395350,3.791343,30389.372963,168.918678,...,0.027340,0.005479,0.467737,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,2,407.704165,319.346951,8.742375e+04,2.956552e+05,24.741866,25.080397,5.403779,9164.199145,185.104354,...,-0.593123,0.210615,0.205048,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,3,594.304200,452.701614,2.620644e+05,9.703912e+05,270.313265,25.855757,10.030686,19491.540312,78.070934,...,0.606505,-0.182648,0.444085,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,4,1022.596612,1047.575478,2.358952e+06,1.676264e+06,274.445483,39.392211,39.249270,50366.540970,70.352124,...,-0.154683,-0.122931,0.751242,0.0,0.0,0.0,0.0,0.0,1.0,0.0
5,5,42.233256,117.546195,3.065312e+06,2.829506e+06,4051.213250,262.075491,122.984911,468051.107518,122.855439,...,-0.619148,-0.184176,1.032192,0.0,0.0,1.0,0.0,0.0,0.0,0.0
6,6,744.939424,696.683758,1.664435e+05,-2.433380e+03,194.082640,23.669475,8.332022,12584.646488,154.997953,...,0.108266,0.448953,0.261249,0.0,0.0,0.0,0.0,0.0,0.0,1.0
7,7,563.084270,1268.059586,2.584751e+06,1.334576e+06,242.013869,85.194208,32.016838,31401.869232,99.300976,...,0.196231,-0.117872,0.983015,0.0,0.0,0.0,0.0,0.0,0.0,1.0
8,8,771.744079,652.637138,2.465070e+06,1.913976e+06,-121.580015,7.729106,11.481541,3145.576053,94.270671,...,0.495622,-0.120732,0.195253,0.0,0.0,0.0,0.0,1.0,0.0,0.0
9,9,1077.459942,1039.630717,3.332191e+06,4.702219e+06,85.343057,-0.742409,9.168484,-8411.855869,79.290742,...,0.514199,-0.586428,0.160507,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Preparing the synthesized data

Although the synthesized dataset is generated based on the data in the raw dataset, some preprocessing is required to make it really alike and ready to train our model. Therefore, we will now first rescale every column using its raw column equivalent. Next, we will round using the same number of decimals used in the raw dataset.

In [6]:
# Load data
synth_data = Dataset.Tabular.from_delimited_files(path=(default_ds, 'steel-plates/steel_plates_synthesized.csv'))
raw_data = Dataset.Tabular.from_delimited_files(path=(default_ds, 'steel-plates/steel_plates_raw.csv'))

# Transform to pandas dataframe
synth_df = synth_data.to_pandas_dataframe().drop(columns=['Column1'])
raw_df = raw_data.to_pandas_dataframe().drop(columns=['Column1'])

In [9]:
def create_min_dict(df):
    """
    Create dictionary of minimum value per column. 
    """
    min_dict = {}
    for column in df.columns:
        min_dict[column] = df[column].min()
    return min_dict


def create_max_dict(df):
    """
    Create dictionary of maximum value per column. 
    Uncomment print statement to track results.
    """
    max_dict = {}
    for column in df.columns:
        max_dict[column] = df[column].max()
#         print(f"{column}: {max_dict[column]}")
    return max_dict


def rescale(value, raw_min, raw_max, synth_min, synth_max):
    x = (value - synth_min)/(synth_max - synth_min)
    new_value = x * (raw_max - raw_min) + raw_min 
    return new_value


def rescale_synth_df(synth_df, raw_df):
    """
    Rescale synthesized dataframe according to raw dataframe. 
    Create min and max dictionaries to use rescale formula.
    """
    raw_min_dict, raw_max_dict = create_min_dict(raw_df), create_max_dict(raw_df)
    synth_min_dict, synth_max_dict = create_min_dict(synth_df), create_max_dict(synth_df)
    synth_df_rescaled = pd.DataFrame()
    for column in synth_df.columns:
        synth_df_rescaled[column] = synth_df[column].apply(
            lambda x: rescale(x, raw_min_dict[column], raw_max_dict[column],
                             synth_min_dict[column], synth_max_dict[column]))
    return synth_df_rescaled


synth_df_rescaled = rescale_synth_df(synth_df, raw_df)
synth_df_rescaled.head() 


# Uncomment to check if min and max values are now similar

# raw_min_dict = create_min_dict(raw_df)
# raw_max_dict = create_max_dict(raw_df)
# synth_min_dict = create_min_dict(synth_df_rescaled)
# synth_max_dict = create_max_dict(synth_df_rescaled)

# print([raw_min_dict[key] for key in raw_min_dict.keys()]) 
# print([synth_min_dict[key] for key in synth_min_dict.keys()]) 
# print([raw_max_dict[key] for key in raw_max_dict.keys()]) 
# print([synth_max_dict[key] for key in synth_max_dict.keys()]) 


[0, 4, 6712, 6724, 2, 2, 1, 250, 0, 37, 1227, 0, 0, 40, 0.0, 0.0, 0.0083, 0.0015, 0.0144, 0.0484, 0.0, 0.301, 0.301, 0.0, -0.991, -0.9989, 0.11900000000000001, 0, 0, 0, 0, 0, 0, 0]
[0.0, 4.0, 6712.0, 6724.0, 2.0, 2.0, 1.0, 250.0, 0.0, 37.0, 1227.0, 0.0, 0.0, 40.0, 0.0, 0.0, 0.0083, 0.0015, 0.0144, 0.0484, 0.0, 0.301, 0.301, 0.0, -0.991, -0.9989, 0.11900000000000001, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[1705, 1713, 12987661, 12987692, 152655, 10449, 18152, 11591414, 203, 253, 1794, 1, 1, 300, 0.9952, 0.9439, 1.0, 0.8759, 1.0, 1.0, 1.0, 5.1837, 3.0741, 4.2587, 0.9917, 0.6421, 1.0, 1, 1, 1, 1, 1, 1, 1]
[1705.0, 1713.0, 12987661.0, 12987692.0, 152655.0, 10449.0, 18152.0, 11591414.0, 203.0, 253.0, 1794.0, 1.0, 1.0, 300.0, 0.9952, 0.9439, 1.0, 0.8759, 1.0, 1.0, 1.0, 5.1837, 3.0741, 4.2587, 0.9916999999999999, 0.6421, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]


In [42]:
def find_decimals(value):
    try:
        decimals = len(str(value).split(".")[1])
    except:
        decimals = 0
    return decimals


def decimals_column(column):
    dec_column = column.apply(find_decimals)
    decimals_list = dec_column.unique().tolist()
    return decimals_list


def create_decimal_dict(df):
    decimal_dict = {}
    for column_name in df.columns:
        column = df[column_name]
        decimals_list = [int(i) for i in decimals_column(column)]
        decimal_dict[column_name] = max(decimals_list)
#         print(f"{column_name}: {decimals_list}, max value: {max(decimals_list)}")
    return decimal_dict


def round_synth_df(synth_df, raw_df):
    """
    Create dictionary of decimal values per column in original df. 
    Then use dictionary to round values in synthesized df and return
    synth_df_rounded.
    In case original data is integer type, change data type.
    """
    decimal_dict = create_decimal_dict(raw_df)
    synth_df_rounded = pd.DataFrame()
    for key in decimal_dict.keys():
        synth_df_rounded[key] = synth_df[key].round(decimal_dict[key])
        if decimal_dict[key] == 0:
            synth_df_rounded[key] = synth_df_rounded[key].apply(lambda x: int(x))
    return synth_df_rounded
    


synth_df_new = round_synth_df(synth_df_rescaled, raw_df)
synth_df_new.head()


# Check if same decimals are used in original df vs synthesized df.

# decimal_dict = create_decimal_dict(raw_df)
# decimal_dict_synth = create_decimal_dict(synth_df_new)
# print([decimal_dict[key] for key in decimal_dict.keys()]) 
# print([decimal_dict_synth[key] for key in decimal_dict_synth.keys()])


,X_Minimum,X_Maximum,Y_Minimum,Y_Maximum,Pixels_Areas,X_Perimeter,Y_Perimeter,Sum_of_Luminosity,Minimum_of_Luminosity,Maximum_of_Luminosity,...,Orientation_Index,Luminosity_Index,SigmoidOfAreas,Pastry,Z_Scratch,K_Scatch,Stains,Dirtiness,Bumps,Other_Faults
0,932,1099,1168151,1770634,6579,531,1581,206534,103,105,...,0.737641,-0.333339,0.737569,0,0,0,0,0,1,0
1,162,321,481231,552107,6462,285,700,161171,162,107,...,0.001325,-0.006773,0.495995,0,0,1,0,0,0,0
2,429,408,339790,455914,4695,390,746,95334,175,139,...,-0.561762,0.184151,0.270089,0,0,0,0,0,0,1
3,589,519,477459,1024852,6302,397,876,127368,88,97,...,0.526934,-0.181867,0.475655,0,0,0,0,0,0,1
4,954,1010,2130438,1620045,6329,530,1697,223138,82,105,...,-0.163866,-0.126286,0.739802,0,0,0,0,0,1,0


In [43]:
synth_df_new.shape

(10000, 34)

In [44]:
# Now we have to divide the data in 'Healthy' vs 'Unhealthy'
# We first check 

synth_df_new['total_fault'] = synth_df_new[['Pastry','Z_Scratch', 'K_Scatch', 'Stains', 'Dirtiness', 'Bumps', 'Other_Faults']].sum(axis=1)
print(f"Maximum faults per steel plate: {synth_df_new['total_fault'].max()}") # check if there's any rows that count multiple faults per steel plate
print(f"Single fault steel plate: {len(synth_df_new[synth_df_new['total_fault'] == 1])}") # check amount of rows with one fault

Maximum faults per steel plate: 3
Single fault steel plate: 8575


In [45]:
# Filter out observations with one fault only
synth_df_new = synth_df_new[synth_df_new['total_fault'] == 1]

In [51]:
print(f"shape: {synth_df_new.shape}")

print(f"Pastry: {synth_df_new.Pastry.sum()}")
print(f"Z_Scratch: {synth_df_new.Z_Scratch.sum()}")
print(f"K_Scatch: {synth_df_new.K_Scatch.sum()}")
print(f"Stains: {synth_df_new.Stains.sum()}")
print(f"Dirtiness: {synth_df_new.Dirtiness.sum()}")
print(f"Bumps: {synth_df_new.Bumps.sum()}")
print(f"Other_Faults: {synth_df_new.Other_Faults.sum()}")

shape: (8575, 35)
Pastry: 825
Z_Scratch: 780
K_Scatch: 1642
Stains: 350
Dirtiness: 128
Bumps: 2007
Other_Faults: 2843


In [95]:
unhealthy_df = synth_df_new[synth_df_new['Stains'] == 1]

steelplate_df = synth_df_new[synth_df_new['Other_Faults'] == 1].append(unhealthy_df)

steelplate_df.shape

(3193, 35)

In [96]:
steelplate_df['Healthy'] = np.where(steelplate_df['Other_Faults'] == 1, 1, 0)
steelplate_df = steelplate_df.drop(columns=['Pastry','Z_Scratch', 'K_Scatch', 'Stains', 'Dirtiness', 'Bumps', 'Other_Faults', 'total_fault'])

steelplate_df.head()

,X_Minimum,X_Maximum,Y_Minimum,Y_Maximum,Pixels_Areas,X_Perimeter,Y_Perimeter,Sum_of_Luminosity,Minimum_of_Luminosity,Maximum_of_Luminosity,...,Edges_X_Index,Edges_Y_Index,Outside_Global_Index,LogOfAreas,Log_X_Index,Log_Y_Index,Orientation_Index,Luminosity_Index,SigmoidOfAreas,Healthy
2,429,408,339790,455914,4695,390,746,95334,175,139,...,0.603528,0.969057,0.0,1.908936,1.093271,1.338372,-0.561762,0.184151,0.270089,1
3,589,519,477459,1024852,6302,397,876,127368,88,97,...,0.818349,0.924140,1.0,2.360965,1.012383,2.439176,0.526934,-0.181867,0.475655,1
6,717,720,402081,204566,5803,376,828,105943,151,157,...,0.638350,0.984341,1.0,1.863360,0.850797,1.407998,0.074768,0.405977,0.318420,1
7,562,1192,2308435,1331933,6117,980,1494,164312,105,95,...,0.286897,0.595485,1.0,3.670246,1.210383,2.153929,0.154598,-0.121578,0.939121,1
14,701,803,1677804,2061597,5309,543,1366,168777,98,89,...,0.379978,0.964895,1.0,2.794868,0.990919,2.516904,0.419589,-0.204210,0.861358,1


In [97]:
# Check highly correlating columns to drop

def get_redundant_pairs(df):
    '''Get diagonal and lower triangular pairs of correlation matrix'''
    pairs_to_drop = set()
    cols = df.columns
    for i in range(0, df.shape[1]):
        for j in range(0, i+1):
            pairs_to_drop.add((cols[i], cols[j]))
    return pairs_to_drop

def get_top_abs_correlations(df, n=5):
    au_corr = df.corr().abs().unstack()
    labels_to_drop = get_redundant_pairs(df)
    au_corr = au_corr.drop(labels=labels_to_drop).sort_values(ascending=False)
    return au_corr[0:n]

print("Top Absolute Correlations")
print(get_top_abs_correlations(steelplate_df, 30))

Top Absolute Correlations
TypeOfSteel_A300       TypeOfSteel_A400         0.935581
X_Minimum              X_Maximum                0.830118
Y_Minimum              Y_Maximum                0.781011
Outside_Global_Index   Orientation_Index        0.766476
Log_Y_Index            SigmoidOfAreas           0.746085
Pixels_Areas           Sum_of_Luminosity        0.744766
X_Perimeter            Sum_of_Luminosity        0.723267
LogOfAreas             SigmoidOfAreas           0.715557
X_Perimeter            Y_Perimeter              0.696950
Pixels_Areas           X_Perimeter              0.696311
Edges_X_Index          Log_Y_Index              0.692858
LogOfAreas             Log_Y_Index              0.664915
Minimum_of_Luminosity  Luminosity_Index         0.662048
Edges_X_Index          SigmoidOfAreas           0.661417
Y_Perimeter            Log_Y_Index              0.641640
Maximum_of_Luminosity  Luminosity_Index         0.641354
Y_Perimeter            Sum_of_Luminosity        0.617315
Pixel

In [98]:
# Drop often occuring highly correlating columns
steelplate_df = steelplate_df.drop(columns=['TypeOfSteel_A400', 'X_Maximum', 'Y_Maximum', 'Sum_of_Luminosity', 'SigmoidOfAreas', 'Y_Perimeter'])

print("Top Absolute Correlations")
print(get_top_abs_correlations(steelplate_df, 10))

Top Absolute Correlations
Outside_Global_Index   Orientation_Index        0.766476
Pixels_Areas           X_Perimeter              0.696311
Edges_X_Index          Log_Y_Index              0.692858
LogOfAreas             Log_Y_Index              0.664915
Minimum_of_Luminosity  Luminosity_Index         0.662048
Maximum_of_Luminosity  Luminosity_Index         0.641354
Edges_Y_Index          Log_X_Index              0.570413
X_Perimeter            LogOfAreas               0.560924
Minimum_of_Luminosity  Maximum_of_Luminosity    0.560354
X_Perimeter            Log_X_Index              0.552984
dtype: float64


## Register training dataset

Now that we have our final dataset ready, it is your task to register it as a Tabular Dataset that can be used in the model training notebook. You may want to use the Microsoft documentation on creating datasets: https://docs.microsoft.com/en-us/azure/machine-learning/how-to-create-register-datasets

In [99]:
steelplate_df.to_csv("steelplate_preprocessed.csv")

default_ds.upload_files(files=['steelplate_preprocessed.csv'], 
                       target_path='pdm-data/', 
                       overwrite=True, 
                       show_progress=True)

# Create Tabular dataset from uploaded csv in datastore
sp_synth_dataset = Dataset.Tabular.from_delimited_files(path=(default_ds, 'pdm-data/steelplate_preprocessed.csv'))


# Register the tabular dataset
try:
    sp_synth_dataset = sp_synth_dataset.register(workspace=ws, 
                                        name='steelplate training dataset',
                                        description='steelplate preprocessed synthesized dataset',
                                        tags = {'format':'CSV'},
                                        create_new_version=True)
except Exception as ex:
    print(ex)

Uploading an estimated of 1 files
Uploading steelplate_preprocessed.csv
Uploaded steelplate_preprocessed.csv, 1 files out of an estimated total of 1
Uploaded 1 files
